In [1]:
!tar zxvf deepcomedy.tar.gz
!tar zxvf data.tar.gz

deepcomedy/
deepcomedy/utils.py
deepcomedy/models/
deepcomedy/models/layers.py
deepcomedy/models/decoder_only.py
deepcomedy/models/transformer.py
deepcomedy/models/__pycache__/
deepcomedy/models/__pycache__/layers.cpython-37.pyc
deepcomedy/models/__pycache__/__init__.cpython-37.pyc
deepcomedy/models/__pycache__/transformer.cpython-37.pyc
deepcomedy/models/__init__.py
deepcomedy/models/.ipynb_checkpoints/
deepcomedy/models/.ipynb_checkpoints/transformer-checkpoint.py
deepcomedy/preprocessing.py
deepcomedy/__pycache__/
deepcomedy/__pycache__/utils.cpython-37.pyc
deepcomedy/__pycache__/__init__.cpython-37.pyc
deepcomedy/__pycache__/preprocessing.cpython-37.pyc
deepcomedy/metrics.py
deepcomedy/__init__.py
deepcomedy/.ipynb_checkpoints/
data/
data/orlando.txt
data/divina_textonly.txt
data/divina.txt
data/divina_syll_textonly.txt
data/orlando-textonly.txt
data/divina_syll.txt
data/.ipynb_checkpoints/
data/.ipynb_checkpoints/orlando-checkpoint.txt
data/.ipynb_checkpoints/orlando-textonly-chec

In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import io
import os
import re
import time
import unicodedata
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import *
from deepcomedy.utils import *

## 1. Data preprocessing

In [2]:
raw_text = open("./data/divina_textonly.txt", "rb").read().decode(encoding="utf-8")
raw_syll_text = (
    open("./data/divina_syll_textonly.txt", "rb").read().decode(encoding="utf-8")
)
syll_text = preprocess_text(raw_syll_text, end_of_tercet='')
text = preprocess_text(raw_text, end_of_tercet='', word_level= True)

Split preprocessed text into verses

In [3]:
sep = "<EOV>"
input_tercets = [x.lstrip() + sep for x in text.split(sep)][:-1]
target_tercets = [x.lstrip() + sep for x in syll_text.split(sep)][:-1]

Encode with input and target tokenizers

In [4]:
input_tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False, filters="", lower=False
)
input_tokenizer.fit_on_texts(input_tercets)

target_tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False, filters="", lower=False
)
target_tokenizer.fit_on_texts(target_tercets)

enc_input_tercets = input_tokenizer.texts_to_sequences(input_tercets)
enc_target_tercets = target_tokenizer.texts_to_sequences(target_tercets)

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [5]:
input_text = []
target_text = []
target_text_tercet = []

for line in range(len(enc_input_tercets) - 2):
    input_text.append(list(chain(*enc_input_tercets[line : line + 3])))
    target_text_tercet.append(list(chain(*enc_target_tercets[line : line + 3])))
    target_text.append(list(chain(*enc_target_tercets[line : line + 4])))

Pad sequences

In [6]:
padded_input_text = tf.keras.preprocessing.sequence.pad_sequences(
    input_text, padding="post"
)
padded_target_text = tf.keras.preprocessing.sequence.pad_sequences(
    target_text, padding="post"
)

In [7]:
input_train, input_val, target_train, target_val = train_test_split(padded_input_text, padded_target_text)

## 2. Hyperparameter sweep

In [52]:
sweep_config = {
    "name": "word2char-gen-sweep-2",
    "method": "grid",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "batch_size": {"value": 32},
        "epochs": {"value": 15},
        "num_layers": {"values": [4, 8, 12]},
        "num_heads": {"values": [4, 8]},
        "d_model": {"value": 256},
        "dff": {"values": [512, 1024]},
    },
}

sweep_id = wandb.sweep(sweep_config, project='deepcomedy', entity='deepcomedy')

Create sweep with ID: ohxcbbae
Sweep URL: https://wandb.ai/deepcomedy/deepcomedy/sweeps/ohxcbbae


In [ ]:
start_symbol = target_tokenizer.word_index["<GO>"]
stop_symbol = target_tokenizer.word_index["<EOV>"]

# Input for generation
encoder_input = [input_text[0]]
decoder_input = [target_text_tercet[0]]


def sweep():
    with wandb.init() as run:
        config = wandb.config
        dataset = make_dataset(input_train, target_train, batch_size=config["batch_size"])
        validation_dataset = make_dataset(input_val, target_val, batch_size=config["batch_size"])
        model, trainer = make_transformer_model(config, input_vocab_size, target_vocab_size, checkpoint_save_path=None)
        trainer.train(dataset, config["epochs"], log_wandb=True, validation_dataset=validation_dataset, validation_every=5)

        result = generate(model, encoder_input, decoder_input, input_tokenizer, target_tokenizer, 27, start_symbol, stop_symbol)
        result = strip_tokens(result)
        result = '<br>'.join(result.split('\n'))
        wandb.log({"generated": wandb.Html("<pre>"+result+"</pre>", inject=False)})
        
wandb.agent(sweep_id, function=sweep)

wandb: Agent Starting Run: jq37nmvy with config:
wandb: 	batch_size: 32
wandb: 	d_model: 256
wandb: 	dff: 512
wandb: 	epochs: 15
wandb: 	num_heads: 4
wandb: 	num_layers: 4
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1 Batch 0 Loss 5.4572 Accuracy 0.0049
Epoch 1 Batch 50 Loss 4.3120 Accuracy 0.1064
Epoch 1 Batch 100 Loss 3.7332 Accuracy 0.1581
Epoch 1 Batch 150 Loss 3.4701 Accuracy 0.1802
Epoch 1 Batch 200 Loss 3.2360 Accuracy 0.2075
Epoch 1 Batch 250 Loss 3.0311 Accuracy 0.2345
Epoch 1 Batch 300 Loss 2.8734 Accuracy 0.2549
Epoch 1 Loss 2.7931 Accuracy 0.2651
Time taken for 1 epoch: 49.21 secs

Epoch 2 Batch 0 Loss 2.0052 Accuracy 0.3713
Epoch 2 Batch 50 Loss 2.0027 Accuracy 0.3675
Epoch 2 Batch 100 Loss 1.9902 Accuracy 0.3687
Epoch 2 Batch 150 Loss 1.9772 Accuracy 0.3708
Epoch 2 Batch 200 Loss 1.9657 Accuracy 0.3728
Epoch 2 Batch 250 Loss 1.9537 Accuracy 0.3758
Epoch 2 Batch 300 Loss 1.9396 Accuracy 0.3793
Epoch 2 Loss 1.9299 Accuracy 0.3816
Time taken for 1 epoch: 40.37 secs

Epoch 3 Batch 0 Loss 1.8242 Accuracy 0.4120
Epoch 3 Batch 50 Loss 1.8062 Accuracy 0.4139
Epoch 3 Batch 100 Loss 1.7871 Accuracy 0.4194
Epoch 3 Batch 150 Loss 1.7680 Accuracy 0.4241
Epoch 3 Batch 200 Loss 1.7503 Accurac

train_loss,0.35586
train_accuracy,0.88242
_runtime,755
_timestamp,1622223280
_step,15
val_loss,0.3506
val_accuracy,0.88783


train_loss,█▆▅▄▄▄▃▃▃▂▂▁▁▁▁
train_accuracy,▁▂▃▄▄▄▅▅▆▇▇████
_runtime,▁▁▂▂▃▃▄▄▅▅▆▆▆▇█
_timestamp,▁▁▂▂▃▃▄▄▅▅▆▆▆▇█
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
val_loss,█▃▁
val_accuracy,▁▆█


wandb: Agent Starting Run: 899qddlr with config:
wandb: 	batch_size: 32
wandb: 	d_model: 256
wandb: 	dff: 512
wandb: 	epochs: 15
wandb: 	num_heads: 4
wandb: 	num_layers: 8
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

Epoch 1 Batch 0 Loss 5.1076 Accuracy 0.0045
Epoch 1 Batch 50 Loss 3.8819 Accuracy 0.1597
Epoch 1 Batch 100 Loss 3.4748 Accuracy 0.1835
Epoch 1 Batch 150 Loss 3.3180 Accuracy 0.1934
Epoch 1 Batch 200 Loss 3.2340 Accuracy 0.1993
Epoch 1 Batch 250 Loss 3.1817 Accuracy 0.2033
Epoch 1 Batch 300 Loss 3.1434 Accuracy 0.2063
Epoch 1 Loss 3.0962 Accuracy 0.2114
Time taken for 1 epoch: 148.63 secs

Epoch 2 Batch 0 Loss 2.5008 Accuracy 0.2810
Epoch 2 Batch 50 Loss 2.4176 Accuracy 0.2850
Epoch 2 Batch 100 Loss 2.3130 Accuracy 0.3060
Epoch 2 Batch 150 Loss 2.2507 Accuracy 0.3188
Epoch 2 Batch 200 Loss 2.2059 Accuracy 0.3270
Epoch 2 Batch 250 Loss 2.1714 Accuracy 0.3336
Epoch 2 Batch 300 Loss 2.1401 Accuracy 0.3393
Epoch 2 Loss 2.1234 Accuracy 0.3421
Time taken for 1 epoch: 120.22 secs

Epoch 3 Batch 0 Loss 1.9642 Accuracy 0.3695
Epoch 3 Batch 50 Loss 1.9353 Accuracy 0.3766
Epoch 3 Batch 100 Loss 1.9200 Accuracy 0.3799
Epoch 3 Batch 150 Loss 1.9038 Accuracy 0.3846
Epoch 3 Batch 200 Loss 1.8842 Accur

train_loss,1.03931
train_accuracy,0.65046
_runtime,2273
_timestamp,1622236327
_step,15
val_loss,1.00302
val_accuracy,0.663


train_loss,█▅▄▃▃▂▃▂▂▂▁▁▁▁▁
train_accuracy,▁▃▄▅▅▆▆▇▇▇▇████
_runtime,▁▁▂▂▃▃▄▄▅▅▆▆▆▇█
_timestamp,▁▁▂▂▃▃▄▄▅▅▆▆▆▇█
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
val_loss,█▃▁
val_accuracy,▁▆█


wandb: Agent Starting Run: 8fn5x3d8 with config:
wandb: 	batch_size: 32
wandb: 	d_model: 256
wandb: 	dff: 1024
wandb: 	epochs: 15
wandb: 	num_heads: 8
wandb: 	num_layers: 4
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1 Batch 0 Loss 4.7300 Accuracy 0.0176
Epoch 1 Batch 50 Loss 3.8634 Accuracy 0.1671
Epoch 1 Batch 100 Loss 3.4633 Accuracy 0.1901
Epoch 1 Batch 150 Loss 3.2294 Accuracy 0.2104
Epoch 1 Batch 200 Loss 2.9998 Accuracy 0.2400
Epoch 1 Batch 250 Loss 2.8227 Accuracy 0.2625
Epoch 1 Batch 300 Loss 2.6922 Accuracy 0.2790
Epoch 1 Loss 2.6257 Accuracy 0.2874
Time taken for 1 epoch: 59.84 secs

Epoch 2 Batch 0 Loss 1.9935 Accuracy 0.3744
Epoch 2 Batch 50 Loss 1.9744 Accuracy 0.3708
Epoch 2 Batch 100 Loss 1.9673 Accuracy 0.3715
Epoch 2 Batch 150 Loss 1.9561 Accuracy 0.3738
Epoch 2 Batch 200 Loss 1.9457 Accuracy 0.3761
Epoch 2 Batch 250 Loss 1.9359 Accuracy 0.3785
Epoch 2 Batch 300 Loss 1.9250 Accuracy 0.3810
Epoch 2 Loss 1.9169 Accuracy 0.3830
Time taken for 1 epoch: 50.34 secs

Epoch 3 Batch 0 Loss 1.8489 Accuracy 0.4095
Epoch 3 Batch 50 Loss 1.8173 Accuracy 0.4109
Epoch 3 Batch 100 Loss 1.7998 Accuracy 0.4151
Epoch 3 Batch 150 Loss 1.7828 Accuracy 0.4200
Epoch 3 Batch 200 Loss 1.7658 Accurac

## 3. Training



In [11]:
dataset = make_dataset(input_train, target_train)
val_dataset = make_dataset(input_val, target_val)

In [10]:
config = {
    "num_layers" : 6,
    "d_model" : 256,
    "num_heads" : 4,
    "dff" : 512,
}

In [11]:
transformer, transformer_trainer = make_transformer_model(config, input_vocab_size, target_vocab_size, checkpoint_save_path=None)

In [36]:
transformer_trainer.train(dataset, 30, validation_dataset=val_dataset, validation_every=1)

Epoch 1 Batch 0 Loss 5.4091 Accuracy 0.0008
Epoch 1 Batch 50 Loss 4.2407 Accuracy 0.1238
Epoch 1 Batch 100 Loss 3.6921 Accuracy 0.1638
Epoch 1 Batch 150 Loss 3.4692 Accuracy 0.1800
Epoch 1 Batch 200 Loss 3.2968 Accuracy 0.1976
Epoch 1 Batch 250 Loss 3.1047 Accuracy 0.2225
Epoch 1 Batch 300 Loss 2.9436 Accuracy 0.2433
Epoch 1 Batch 0 Validation Loss 1.9496 Validation Accuracy 0.3826
Epoch 1 Batch 50 Validation Loss 1.9754 Validation Accuracy 0.3752
Epoch 1 Batch 100 Validation Loss 1.9726 Validation Accuracy 0.3754
Epoch 1 Loss 2.8584 Accuracy 0.2542
Time taken for 1 epoch: 96.52 secs

Epoch 2 Batch 0 Loss 2.0470 Accuracy 0.3592
Epoch 2 Batch 50 Loss 2.0151 Accuracy 0.3640
Epoch 2 Batch 100 Loss 1.9988 Accuracy 0.3661
Epoch 2 Batch 150 Loss 1.9869 Accuracy 0.3678
Epoch 2 Batch 200 Loss 1.9747 Accuracy 0.3702
Epoch 2 Batch 250 Loss 1.9627 Accuracy 0.3725
Epoch 2 Batch 300 Loss 1.9501 Accuracy 0.3754
Epoch 2 Batch 0 Validation Loss 1.8128 Validation Accuracy 0.4117
Epoch 2 Batch 50 Valida

KeyboardInterrupt: ignored

## 4. Generation

In [50]:
def evaluate(
    transformer,
    encoder_input,
    decoder_input,
    stop_symbol,
    max_length=200,
):
    """
    Predicts the output of the model given the `input_sequence`.
    The `input_sequence` is encoded by the Encoder, then its output is fed to the Decoder,
    whose output is fed back into the Decoder until the `stop_symbol` token is produced.

    This function works with a batch of inputs and stops when all outputs include a stop symbol.
    """

    output = decoder_input

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output
    )

    enc_output = transformer.encoder(
        encoder_input, False, enc_padding_mask
    )  # (batch_size, inp_seq_len, d_model)

    for _ in range(max_length):

        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output
        )

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, _ = transformer.decoder(
            output, enc_output, False, combined_mask, dec_padding_mask
        )

        predictions = transformer.final_layer(dec_output)

        # select the last character from the seq_len dimension
        predicted_ids = tf.argmax(predictions[:, -1:, :], axis=-1)

        # concatenate the predicted_id to the output which is given to the decoder as its input.
        output = tf.concat(
            [
                tf.cast(output, dtype=tf.int32),
                tf.cast(predicted_ids, dtype=tf.int32),
            ],
            axis=-1,
        )
                
        if sum(output.numpy()[0] == stop_symbol) == 4:
            return output

    return output

In [28]:
def generate(transformer, input_sequence, target_sequence, input_tokenizer, target_tokenizer, steps, start_symbol, stop_symbol):

    result = target_tokenizer.sequences_to_texts(target_sequence)[0]
    
    encoder_input = input_sequence
    decoder_input = target_sequence

    for _ in range(steps):

        encoder_input = tf.convert_to_tensor(encoder_input)
        decoder_input = tf.convert_to_tensor(decoder_input)
        output = evaluate(transformer, encoder_input, decoder_input, stop_symbol)

        generated_text = target_tokenizer.sequences_to_texts(output.numpy())[0]
        
        verses = [line.lstrip() + '<EOV> ' for line in generated_text.split('<EOV>') if line.strip() != '']
        
        result = ''.join([result, verses[-1]])
                
        verses = ''.join(verses[-3:])
        
        decoder_input = target_tokenizer.texts_to_sequences([verses])
        
        verses = remove_syll_token(verses)
        verses = re.sub(r"[ ]+", "", verses)
        verses = re.sub("<[^>]*>", " \g<0> ", verses)
        verses = re.sub("<EOV>  <GO>", "<EOV> <GO>", verses)
        verses = verses.strip()

        encoder_input = input_tokenizer.texts_to_sequences([verses])
        
    return result

In [42]:
start_symbol = target_tokenizer.word_index["<GO>"]
stop_symbol = target_tokenizer.word_index["<EOV>"]

encoder_input = [input_text[0]]
decoder_input = [target_text_tercet[0]]

result = generate(transformer, encoder_input, decoder_input, input_tokenizer, target_tokenizer, 10, start_symbol, stop_symbol)

Stopped
Stopped
Stopped
Stopped
Stopped
Stopped
Stopped
Stopped
Stopped
Stopped


In [44]:
print(strip_tokens(result))

|Nel |mez|zo |del |cam|min |di |no|stra |vi|ta
|mi |ri|tro|vai |per |u|na |sel|va o|scu|ra,
|ché |la |di|rit|ta |via |e|ra |smar|ri|ta.
|E |io |a |la |mia |di|stin|ta |di |so|pra
|e |dis|si:« |Vien |che |son |di |so|pra |se|sta
|la |mia |se|gui|ta |di |mia |se|gui|ta,
|che |di |mil|le |mia |di |mi|se|ria |sciol|ta.
|E |io |mi |fui |di |mil|le |di |mil|le |schia|mi
|di |quel |che |di |mio |di |mil|le |si |mo|ve
|di |mil|le |di |mil|le |di |mil|le |spi|ri.
|E |io |dis|si:« |Vien |che |tu |di |mil|le |spal|le
|di |quel |che |di |mio |di |mil|le |si |spi|ri
|di |mil|le |di |mil|le |di |mil|le |spa|da,


## 5. Syllabification

In [ ]:
start_symbol = target_tokenizer.word_index["<GO>"]
stop_symbol = target_tokenizer.word_index["<EOV>"]

In [ ]:
encoder_input = tf.convert_to_tensor([input_text[0]])
decoder_input = tf.convert_to_tensor([[start_symbol]])

In [ ]:
syll_output = evaluate(transformer, encoder_input, decoder_input, stop_symbol, max_length=400)

Stopped


In [ ]:
print(target_tokenizer.sequences_to_texts(syll_output.numpy()))

['<GO> | c h e <SEP> | d i <SEP> | p e n | s i e r <SEP> | m i <SEP> | s t a | v a <SEP> i n <SEP> | u | n o <SEP> | s t r a | l e , <EOV> <GO> | e <SEP> | d i | c o <SEP> | d i <SEP> | g e n | t e <SEP> a l | t r o <SEP> | c h e <SEP> | p i ù <SEP> | d o l | c e » . <EOV> <GO> | N o i <SEP> | e | r a | v a m <SEP> | n e l <SEP> | s u o <SEP> | a | s p e t | t o <SEP> | b a n | d o <EOV> <GO> | c h e <SEP> | l ’ <SEP> a | n i | m a <SEP> | s u a <SEP> | a v | v e n | t a <SEP> | d i | s t a n | t e , <EOV>']


Potrebbe essere underfitting?

## 6. Save model

In [ ]:
transformer.save_weights('models/w2c-gen.h5')

In [ ]:
new_transformer = Transformer(
        num_layers=config["num_layers"],
        d_model=config["d_model"],
        num_heads=config["num_heads"],
        dff=config["dff"],
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        pe_input=1000,
        pe_target=1000,
        rate=0.1,
    )

In [ ]:
# In order to load the new weights the model should be called once for the variables to be initialized

# Any inp, tar is ok here
inp = tf.convert_to_tensor([[start_symbol]])
tar = tf.convert_to_tensor([[start_symbol]])

enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)

new_transformer(inp, tar, False, enc_padding_mask, look_ahead_mask, dec_padding_mask);

In [ ]:
new_transformer.load_weights('models/w2c-gen.h5')

In [ ]:
encoder_input = [input_text[0]]
decoder_input = [target_text_tercet[0]]

result = generate(new_transformer, encoder_input, decoder_input, input_tokenizer, target_tokenizer, 6, start_symbol, stop_symbol)

Stopped
Stopped
Stopped
Stopped
Stopped
Stopped


In [ ]:
result

'<GO> | N e l <SEP> | m e z | z o <SEP> | d e l <SEP> | c a m | m i n <SEP> | d i <SEP> | n o | s t r a <SEP> | v i | t a <EOV> <GO> | m i <SEP> | r i | t r o | v a i <SEP> | p e r <SEP> | u | n a <SEP> | s e l | v a <SEP> o | s c u | r a , <EOV> <GO> | c h é <SEP> | l a <SEP> | d i | r i t | t a <SEP> | v i a <SEP> | e | r a <SEP> | s m a r | r i | t a . <EOV><GO> | E l | l a <SEP> | s o | p r a <SEP> | c h e <SEP> ’ l <SEP> | v i | s o <SEP> a <SEP> | q u e l | l a <SEP> | g e n | t e <EOV> <GO> | c h e <SEP> | p e r <SEP> | l o <SEP> | s u o <SEP> | a v | v e r | s a | r i o <SEP> a l | t r u i <SEP> | m a n | t o , <EOV> <GO> | e <SEP> | a l | t r a <SEP> | v o | c e <SEP> | m i <SEP> | p a | r e a <SEP> | p i ù <SEP> | r a t | t a . <EOV> <GO> « <SEP> | O <SEP> | t u <SEP> | c h e <SEP> | s e ’ <SEP> | c h e <SEP> | s ì <SEP> | p r e s | s o <SEP> | d i | s c i o l | t a » , <EOV> <GO> | d i s | s e <SEP> ’ l <SEP> | m a | e | s t r o , « <SEP> | q u a n | t o <SEP> | p o s | s o 